In [ ]:
#데이터 불러오기
# 전처리한 데이터 불러오기
import pandas as pd

train_df = pd.read_csv('C://Users//and_c//Desktop//sparta_ga4//train_df3.csv', low_memory = False)
df = train_df.copy()

In [ ]:
# 데이터프레임에 한 열의 값을 바꾸기 위해서 def함수를 만들고 apply 적용
import pandas as pd
import datetime #날짜와 시간을 다루는 라이브러리
import pytz #다양한 시간대를 처리하기 위해 사용하는 라이브러리

# Posix 시간을 utc 시간으로 변환하는 함수
def posix_to_utc(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime)
    return utc_time

# Posix 시간을 UTC 지정한 로컬 시간대로 변환하는 함수
def posix_to_utc_local(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime) #posix -> utc시간대로
    local_timezone = pytz.timezone('America/New_York')  # 지역 시간대
    return utc_time.astimezone(local_timezone) # 그 지역 시간대로 변환

In [ ]:
df['visitStartTime'] = df['visitStartTime'].apply(posix_to_utc)

In [ ]:
# date칼럼 추가
df['date'] = df['visitStartTime'].dt.date
# type time으로 변경
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# df 샘플링
df_2016 = df[(df['date'] > '2016-11-01') & (df['date'] < '2017-01-01')]

In [ ]:
# week기준으로 구분한 칼럼 추가
df_2016['period_week'] = df_2016['visitStartTime'].dt.to_period('W').apply(lambda x: x.start_time.strftime('%Y-%m-%d'))

In [ ]:
# user의 첫 방문날짜 칼럼 추가
df_2016['cohort_week'] = df_2016.groupby('fullVisitorId')['period_week'].transform('min')

In [ ]:
df_2016['period_week'] = pd.to_datetime(df_2016['period_week'])
df_2016['cohort_week'] = pd.to_datetime(df_2016['cohort_week'])

In [ ]:
# Cohort_index 구하기 ...
df_2016['cohort_index'] = (df_2016['period_week'] - df_2016['cohort_week'])/7

In [ ]:
# cohort index 0 days에서 0 숫자만 추출
df_2016['cohort_index'] = df_2016['cohort_index'].dt.days

In [ ]:
# cohort에 따른 user수 구하기
cohort_table_2016 = df_2016.groupby(['cohort_week','cohort_index'])['fullVisitorId'].apply(pd.Series.nunique).reset_index()

In [ ]:
# create Table
cohort_pivot_2016 = cohort_table_2016.pivot(index = 'cohort_week', columns=['cohort_index'], values='fullVisitorId')

In [ ]:
# 각 기준이 되는 값(1열)으로 나눠주기
div_cohort_pivot_2016 = cohort_pivot_2016.divide(cohort_pivot_2016.iloc[:,0],axis=0)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# index 수정
div_cohort_pivot_2016.index = div_cohort_pivot_2016.index.strftime('%Y-%m-%d')

# heatmap
plt.figure(figsize=(21,10))
sns.heatmap(div_cohort_pivot_2016, annot = True, cmap='Blues',fmt = '.0%')
plt.title('Weekly Retention', fontsize=20)
plt.xlabel('week', fontsize=14)
plt.ylabel('visit', fontsize=14)